<a href="https://colab.research.google.com/github/ozloty06/PySpark_Amazon-Vine-Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.2 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-14 04:00:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.18MB/s    in 0.2s    

2021-08-14 04:00:52 (6.18 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Vine-Challange").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Extract the dataset used in Deliverable 1
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [5]:
# Recreate vine table and analysis from Deliverable 1
# Clean up tables
dropna_df=df.dropna()
dropna_df=df.distinct()
dropna_df.show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [6]:
dropna_df.count()

1785997

In [8]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
dropna_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26197704|R10010D6B1QM7U|B004HX5OFW|       8155204|Xbox 360 Slim AC ...|     Video Games|          1|            0|          0|   N|                Y|                 POS|I purchased this ...| 2013-03-28|
|         US|   10409381|R100BI61LIKH2N|B000QW9D14|     583724133|Blue Dragon - Xbo...|     Video Games|          5|    

In [9]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = dropna_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(15)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R10010D6B1QM7U|          1|            0|          0|   N|                Y|
|R100BI61LIKH2N|          5|            1|          2|   N|                Y|
|R100TPJCYDYGBU|          3|            1|          2|   N|                Y|
|R1018OO9U6UUWO|          5|            0|          0|   N|                Y|
|R102YIWZEIAXT9|          5|            2|          4|   N|                N|
|R1034S7FA21OI1|          5|            2|          2|   N|                Y|
|R103LC8CTAHWZ6|          2|            1|          1|   N|                Y|
|R103PT7S4HRIP8|          2|            0|          0|   N|                Y|
|R104RCQZYPYDXS|          5|            7|          8|   N|                N|
|R105KZIXHZW63W|          5|            1|          1|   N|     

In [13]:
# Step 1: Filter/create DataFrame where total_votes count >= 20 
vine_total_votes_df= vine_df.filter("total_votes>=20").select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_total_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|
|R2NPZ79NDZ87XO|          1|            1|         31|   N|                N|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [32]:
# check datatypes
# vine_total_votes_df.dtypes
# Step 2: Filter/create DataFrame where (helpful_votes/total_votes) => 50%
from pyspark.sql.functions import col
vine_percents_df=vine_total_votes_df.withColumn('percent', col("helpful_votes")/ col("total_votes"))
vine_percents_df=vine_percents_df.filter('percent>=0.5')
vine_percents_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|0.9148936170212766|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|0.9347826086956522|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|0.8363636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|0.5714285714285714|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|             0.905|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|0.9148936170212766|
|R1TBFM7966BSCZ|          1|           33|         56|   N|                Y|0.5892857142857143|
|R3NSDYMPN0WV6M|          4|  

In [35]:
# Step 3: Filter/create "paid" DataFrame where vine == 'Y'
vine_Y_df=vine_percents_df.filter(col("vine").contains("Y"))
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R24PS9N276OXF4|          5|          241|        262|   Y|                N|0.9198473282442748|
|R3EPHK0JGQGE91|          5|           21|         28|   Y|                N|              0.75|
|R3T4LTJ189G0LT|          4|           37|         45|   Y|                N|0.8222222222222222|
| RKG407AVAZXBQ|          3|           22|         25|   Y|                N|              0.88|
| RNU8PK609WT6P|          4|          347|        362|   Y|                N|0.9585635359116023|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|0.7692307692307693|
|R23WY68NT94WKE|          5|          125|        143|   Y|                N|0.8741258741258742|
|R3JT907UUA1WUI|          4|  

In [36]:
# Step 4: Filter/create "unpaid" DataFrame where vine == 'N'
vine_N_df=vine_percents_df.filter(col("vine").contains("N"))
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|           percent|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R26Q7L4FI2L0R0|          1|           43|         47|   N|                Y|0.9148936170212766|
| R8FTCCXCMWLYT|          1|           43|         46|   N|                Y|0.9347826086956522|
|R350PLZT86NQJR|          4|           46|         55|   N|                Y|0.8363636363636363|
|R278LLN5FZIGPU|          1|           24|         42|   N|                N|0.5714285714285714|
|R3VNNA3YL48039|          5|          181|        200|   N|                Y|             0.905|
| RZAE1OYOXEOAR|          5|           86|         94|   N|                Y|0.9148936170212766|
|R1TBFM7966BSCZ|          1|           33|         56|   N|                Y|0.5892857142857143|
|R3NSDYMPN0WV6M|          4|  

In [62]:
# Step 5: Determine total reviews, 5-star reviews, and percentage of 5-star reviews for both Dataframes
# 5a: count of all rows
vine_Y_df.count()

94

In [63]:
# 5a: count of all rows
vine_N_df.count()

40471

In [64]:
# 5b: count of star_rating = 5
vine_Y_df.filter(vine_Y_df.star_rating == 5).count()

48

In [65]:
# 5b: count of star_rating = 5
vine_N_df.filter(vine_N_df.star_rating == 5).count()

15663

In [66]:
# 5c: percent of all rows with a 5 star_rating
(vine_Y_df.filter(vine_Y_df.star_rating == 5).count())/(vine_Y_df.count())

0.5106382978723404

In [67]:
# 5c: percent of all rows with a 5 star_rating
(vine_N_df.filter(vine_N_df.star_rating == 5).count())/(vine_N_df.count())

0.38701786464381904